In [1]:
import pandas as pd
import tqdm
from datetime import datetime

import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
def rename_columns(comments_df):
    if "text" in list(comments_df.columns):
        comments_df = comments_df.rename(columns={"text": "message"})
    if "id" in list(comments_df.columns):
        comments_df = comments_df.rename(columns={"id": "message_id"})
    
    # comments_df.to_csv(f"{save_dir}/{name}_renamed.csv", index=False)
    return comments_df

In [3]:
def unix_to_month_year(unix_timestamp, type):

    dt = datetime.fromtimestamp(unix_timestamp)
    month = dt.month
    year = dt.year
    if type == "month":
        return month
    elif type == "year":
        return year
    elif type == "month_year":
        return f"{month}_{year}"

In [4]:
def csv_to_sql_table_func(filepath, USERNAME, DATABASE):
    df = pd.read_csv(filepath)
    df = rename_columns(df)
    df = df.dropna(subset=["message", "message_id", "parent_comment_id"])
    df['month_year'] = df['create_time'].apply(lambda x: unix_to_month_year(x, "month_year"))
    
    engine = create_engine(f"mysql://{USERNAME}@localhost/{DATABASE}?charset=utf8mb4")
    df.to_sql(f"cmt{COMMENT_VERSION}", con=engine, index=False, if_exists='replace')
    
    index_query = f"""CREATE INDEX idx_cmt{COMMENT_VERSION}_message ON cmt{COMMENT_VERSION} (message);
    CREATE INDEX idx_cmt{COMMENT_VERSION}_video_id ON cmt{COMMENT_VERSION} (video_id);
    CREATE INDEX idx_cmt{COMMENT_VERSION}_message_id ON cmt{COMMENT_VERSION} (message_id);
    CREATE INDEX idx_cmt{COMMENT_VERSION}_month_year ON cmt{COMMENT_VERSION} (month_year);"""
    
    with engine.connect() as connection:
        connection.execute(text(index_query))
        print(f"Table created and indexed; {len(df)} rows")


In [5]:
COMMENT_VERSION = 7
USERNAME = "iasamori"
DATABASE = "iasamori"

filepath = f"./data/combined_comments_{COMMENT_VERSION}.csv"
csv_to_sql_table_func(filepath, USERNAME, DATABASE)

/tmp/ipykernel_518965/1937413155.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


Table created and indexed; 938852 rows


In [6]:
video_df = pd.read_csv("./data/all_months_2.csv")
video_df = video_df[["id", "create_time", "video_description"]]
video_df

,id,create_time,video_description
0,7063441153508805934,1644585553,#lossofachild #AlaskaAirCAREoke #fentanylaware...
1,7063438031700905262,1644584826,codeine bad don’t do codeine
2,7063399327305354542,1644575823,"You don’t know me, you just know what I post #..."
3,7063351253950008622,1644564622,There has been an 85% increase in fentanyl-rel...
4,7063331793835068719,1644560090,#wedorecover #God #AlaskaAirCAREoke #jesus #So...
...,...,...,...
97452,7207155741814312234,1678046723,#artblindbox #Survivor #agelikevampires so mu...
97453,7207155173251140907,1678046616,Morning Tiktok i can say this filter is amazin...
97454,7207151815496928558,1678045809,#PhysicalTrauma #PhysicalAssault #Opoidwithdra...
97455,7207150205014199595,1678045431,BBJ not a day goes by that I don't miss you. E...


In [7]:
video_df['month_year'] = video_df['create_time'].apply(lambda x: unix_to_month_year(x, "month_year"))
video_df

,id,create_time,video_description,month_year
0,7063441153508805934,1644585553,#lossofachild #AlaskaAirCAREoke #fentanylaware...,2_2022
1,7063438031700905262,1644584826,codeine bad don’t do codeine,2_2022
2,7063399327305354542,1644575823,"You don’t know me, you just know what I post #...",2_2022
3,7063351253950008622,1644564622,There has been an 85% increase in fentanyl-rel...,2_2022
4,7063331793835068719,1644560090,#wedorecover #God #AlaskaAirCAREoke #jesus #So...,2_2022
...,...,...,...,...
97452,7207155741814312234,1678046723,#artblindbox #Survivor #agelikevampires so mu...,3_2023
97453,7207155173251140907,1678046616,Morning Tiktok i can say this filter is amazin...,3_2023
97454,7207151815496928558,1678045809,#PhysicalTrauma #PhysicalAssault #Opoidwithdra...,3_2023
97455,7207150205014199595,1678045431,BBJ not a day goes by that I don't miss you. E...,3_2023


In [8]:
video_df.to_csv("./data/all_months_2_my.csv", index=False)